In [12]:
# clear all the variables
%reset -f

import os
import IPython as ip
import logging

logger = logging.getLogger('scrapy-tutorial')
logger.setLevel(logging.INFO)

In [8]:
# import the relevant libraries
import scrapy
import re

from bs4 import BeautifulSoup
from scrapy.http import FormRequest, Request
from scrapy.crawler import CrawlerProcess 

# for debugging, uncomment the following line
#from IPython.core.debugger import Tracer as tr
# to call trace, tr()()

In [9]:
# the connect spider definition
class loginSpider(scrapy.Spider):
    name = 'kaggle_login'
    start_urls = ['https://www.kaggle.com/account/login?']
    login = 'foofoo@foo.edu'
    #login = 'vermosen.jm@free.fr'
    pwd = 'mypass'
    result = []
    
    def parse(self, response):                
        
        #self.debugPrint(response, filename='initial', append=False)
        cookies = response.meta.setdefault('cookie_jar', CookieJar())
        cookies.extract_cookies(response, response.request)
        
        token = response.css('input[name=__RequestVerificationToken]::attr(value)').extract_first()
        
        return [FormRequest.from_response(response, 
                                          formdata = {
                                              'UserName' : self.login, 
                                              'Password': self.pwd,
                                              '__RequestVerificationToken' : token
                                              }, 
                                          callback = self.after_login)]
    
    def after_login(self, response):
        
        self.debugPrint(response, filename='return', append=False)
        if 'The username or password provided is incorrect.' in response.body:
            logging.error('Login failed')
            return
        else:
            logging.info('Login succeed')

            return Request('https://github.com/' + self.login + '?tab=repositories',
                           callback=self.parse_repo)
            
    def parse_repo(self, response):
        
        res = response.meta['cookiejar']
        #self.debugPrint(response.body)
        # get the repo list
        soup = BeautifulSoup(response.text, 'lxml')
        list = soup.find_all('li', {'class' : 'col-12 d-block width-full py-4 border-bottom public source'})
        list = [l.find_all('div', {'class' : 'd-inline-block mb-1'})[0] for l in list]
        list = [l.find_all('h3')[0] for l in list]
        list = [l.find_all('a')[0].text for l in list]

        [loginSpider.result.append(re.sub('\s+',' ',l).encode('utf-8')) for l in list]
        
    def debugPrint(self, message, filename = 'debug', append = False):
        opt = None; 
        if append:
            opt = 'a'
        else:
            opt = 'wb'
            
        with open('C:\\Temp\\' + filename + '.txt', opt) as f:
            f.write(message.body)
            f.close()

In [ ]:
# set the proxy
os.environ["http_proxy"] = "http://ip:port"
os.environ["https_proxy"] = "https://ip:port"

# set the logging policy
logger = logging.getLogger('scrapy-tutorial')
logger.setLevel(logging.INFO)

process = CrawlerProcess({
'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

sp = loginSpider()
process.crawl(sp)
process.start(stop_after_crawl=True)

print('repo found: {0}'.format(loginSpider.result))

In [12]:
# terminate the kernel
app = ip.Application.instance()
app.kernel.do_shutdown(restart=False)

{'restart': False, 'status': 'ok'}